# 1. Load Packages Required

In [1]:
from datetime import datetime
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Activation, Dense
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from tensorflow.keras.layers import concatenate
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
#from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Activation, Dense, Dropout
from kerastuner.applications import HyperResNet
from kerastuner.tuners import Hyperband
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold

import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import msoffcrypto
import tensorflow as tf
import keras_tuner as kt
import xgboost

<ipython-input-1-59739833d370>:21: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.applications import HyperResNet


# 3. Load Raw Data

In [ ]:
# Change to own directory
# filenames just my files
# just my dataset ignore

df_list = list()
    
raw = pd.concat(df_list)

In [3]:
data = raw

In [ ]:
data.shape

In [ ]:
data.head()

# 6. Functions for LSTM

## Stationarity Test Functions

In [14]:
def adt_check(ts):
    # Store Stationary/Non Stationary for 3 cases, Normal, Detrended and Lagged(Differenced)
    patterns = []
    #data = [ts.data, ts.z_data.dropna(), ts.zp_data.dropna()]
    data = [ts.data]
    for i in data:
        dftest = adfuller(i, autolag='AIC')
        if dftest[4]['5%'] < dftest[0]:
            patterns.append([0,1])
        else:
            patterns.append([1,0])
    return patterns
    
def kpss_check(ts):
    # Store Stationary/Non Stationary for 3 cases, Normal, Detrended and Lagged(Differenced)
    patterns = []
    data = [ts.data]
    for i in data:
        dftest = kpss(ts.data, regression='c')
        if dftest[3]['5%'] < dftest[0]:
            patterns.append([0,1])
        else:
            patterns.append([1,0])
    return patterns
    
def stationary_check(ts):
    ts['z_data'] = np.log(ts['data'])
    ts['zp_data'] = ts['z_data'] - ts['z_data'].shift(1)
    adt_status = adt_check(ts)
    kpss_status = kpss_check(ts) 
    patterns = [x + y for x, y in zip(adt_status, kpss_status)]
    # If both stationary
    if patterns[0] == [1, 0, 1, 0]:
        return 'stationary'
    elif patterns[0] == [0, 1, 0, 1]:
        return 'non_stationary'
    else:
        # ADF - Stationary and KPSS - Non Stationary
        if patterns[0][0] == 1 and patterns[0][3] == 1:
          return 'trend'
        elif patterns[0][1] == 1 and patterns[0][2] == 1:
          return 'difference'

In [17]:
def log_transform(data, inverse=False):
  if inverse: 
    return np.exp(data) + 1
  return np.log(data + 1)

def trend_transform(data, inverse=False):
  if inverse:
    return (pd.DataFrame(data) * pd.DataFrame(data).rolling(window=12).std()) + pd.DataFrame(data).rolling(window=12).mean()
  return (data - data.Cases.rolling(window=12).mean()) / data.Cases.rolling(window=12).std()

def difference_transform(data, inverse=False):
  if inverse:
    shifted = data[0] + data[:-1]
    return  [x + y for (x, y) in zip(data, shifted)]
  return data - data.shift(1)

## LSTM Model Functions

In [15]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 

def build_model(hp):
    model = Sequential(LSTM(hp.Int('input_unit', min_value=10, max_value=60, step=10),
                                  return_sequences=True,
                                  input_shape=(train_X.shape[1], train_X.shape[2])))
    for i in range(hp.Int('n_layers', 1,4)):
        model.add(LSTM(hp.Int(f'lstm_{i}_units', min_value=10, max_value=60, step=10),
                      return_sequences=True))
    
    model.add(Dropout(0.5))
    
    model.add(Bidirectional(LSTM(hp.Int('input_unit', min_value=10, max_value=60, step=10),
                                  input_shape=(train_X.shape[1], train_X.shape[2]))))
    
    model.add(Dense(1, activation=hp.Choice('dense_activation',values=['relu', 'sigmoid'],default='relu')))
    model.compile(loss='mean_squared_error', optimizer='adam',metrics = ['mse'])
    
    return model

In [14]:
def loss_plots(model, train_X, train_y, test_X, test_y):
    history = model.fit(train_X, train_y, 
                        epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    title = 'placeholder' + '.png'
    pyplot.savefig(title)

# 7. Run Models

In [16]:
def lstm_model(train_X, test_X, train_y, test_y):
    tuner = kt.RandomSearch(build_model,
                                objective='val_loss',
                                max_trials=10,
                                overwrite = True)

    tuner.search(train_X,train_y, validation_data=(test_X, test_y),
                      epochs=50, batch_size=72)

    fs_model = tuner.get_best_models(num_models=1)[0]
    history = fs_model.fit(train_X, train_y, 
                                  epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)


    yhat = fs_model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)


    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    return inv_y, inv_yhat


# fit an xgboost model and make a one step prediction
def xgboost_forecast(train, testX):
    # transform list into array
    train = np.asarray(train)
    # split into input and output columns
    trainX, trainy = train[:, :-1], train[:, -1]
    # fit model
    model = xgboost.XGBRegressor(objective='reg:squarederror', n_estimators=1000)
    trainX = np.ascontiguousarray(trainX)
    trainy = np.ascontiguousarray(trainy)
    model.fit(trainX, trainy)
    # make a one-step prediction
    testX = np.array([testX])
    testX = np.ascontiguousarray(testX)
    yhat = model.predict(testX)
    return yhat[0]

# walk-forward validation for multivariate data
def walk_forward_validation(train ,test):
    predictions = list()
    test_X, test_y = test[:, :-1], test[:, -1]
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model on history and make a prediction
        yhat = xgboost_forecast(history, test_X[i,:])
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i,:])
        # summarize progress
#         print('>expected=%.1f, predicted=%.1f' % (test_y[i], yhat))

    test_X = test_X.reshape((test_X.shape[0], test_X.shape[1]))
    yhat = np.array(predictions)
    yhat = yhat.reshape((len(yhat),1))
    inv_yhat = np.hstack((yhat, test_X[:, 1:]))

    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]
    test_y = test_y.reshape((len(test_y), 1))
    inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]
    return inv_y, inv_yhat


In [17]:
def add_dates(df):
    y = np.array(df['Year']-1970, dtype='<M8[Y]')
    m = np.array(df['Month']-1, dtype='<m8[M]')
    d = np.array(0, dtype='<m8[D]')
    dates = pd.Series(y+m+d)
    
    df.insert(len(df.columns), 'date', dates)
    df = df.iloc[: , 4:]
    df.set_index('date', inplace=True)
    df.index.name = 'date'
    return df

In [18]:
def missing_data(df, targeted_length):
    
    curr_length = len(df)

    if curr_length != targeted_length:
        df_2016_03 = df[(df['Year'] == 2016) & (df['Month'] == 4)]

        if len(df_2016_03) != 0 and (curr_length + 1) == targeted_length:
            df_2016_03 = df_2016_03.reset_index()
            df_2016_03.loc[0, 'Month'] = 3

            frames = [df, df_2016_03]
            df_final = pd.concat(frames)
            df_final = df_final.reset_index(drop=True)
            df_final = df_final.sort_values(['Year', 'Month'], ascending=[True, True])

        else:
            mux = pd.MultiIndex.from_product([df['placeholder'].unique(),
                                      range(2015,2021),
                                      range(1, 13)], names=['placeholder','Year','Month'])

            df_final = df.set_index(['placeholder','Year','Month']).reindex(mux, fill_value=0).reset_index()
    
    result = df_final
    
    return result

In [19]:
# Define Constants
variables = ['var1(t)','var2(t)', 'var3(t)', 'var4(t)', 'var5(t)',
               'var6(t)', 'var7(t)', 'var8(t)', 'var9(t)', 'var10(t)']

In [ ]:
for i in range(len(placeholder)):
    curr_cat = placeholder[i]
    category_data = []
    curr_drop = 'var' + str(i+1) + '(t)'
    curr_var = list(variables)
    if curr_drop in curr_var:
        curr_var.remove(curr_drop)
    print(curr_cat)
        
    for j in range(len(placeholder)):
        curr_holder = placeholder[j]
        curr_record = []
        curr_record.append(curr_holder)
        df_test = curr_df.loc[curr_df['placeholder'] == curr_holder]
        df_inter = missing_data(df_test, 72)
        df_final = add_dates(df_inter)
        df_final = df_final.astype(int)

        # Running LSTM
        # load dataset
        values = df_final.values

        # ensure all data is float
        values = values.astype('float32')

        # scale data
        scaler = MinMaxScaler(feature_range=(0, 3))
        scaled = scaler.fit_transform(values)

        # frame as supervised learning
        reframed = series_to_supervised(scaled, 1, 1)

        reframed.drop(reframed[curr_var], axis=1, inplace=True)

        # split into train and test sets
        values = reframed.values
        n_train_hours = 56
        train = values[:n_train_hours, :]
        test = values[n_train_hours:, :]

         # split into input and outputs
        train_X, train_y = train[:, :-1], train[:, -1]
        test_X, test_y = test[:, :-1], test[:, -1]

        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
        test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

        inv_y, inv_yhat = lstm_model(train_X, test_X, train_y, test_y)
        lstm_rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
        lstm_mae = mean_absolute_error(inv_y, inv_yhat)
        
        curr_record.extend((lstm_rmse, lstm_mae))

        # xgboost 
        inv_y, inv_yhat = walk_forward_validation(train, test)
        xgboost_rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
        xgboost_mae = mean_absolute_error(inv_y, inv_yhat)
        
        curr_record.extend((xgboost_rmse, xgboost_mae))
        
        category_data.append(curr_record)
    
    output = pd.DataFrame(category_data, columns=['Placeholder', 'LSTM_RMSE', 'LSTM_MAE',
                                                 'XGBoost_RMSE', 'XGBoost_MAE'])
    curr_filename = '/Users/admin/Desktop/Y4S1/BT4103/' + curr_cat + '.csv'
    output.to_csv(curr_filename, index=False)
    print('---')
